# Tutorial 5: Series Temporales con TDA

## Análisis Topológico de Señales EEG y fMRI

**Autor:** MARK-126  
**Nivel:** Avanzado  
**Tiempo estimado:** 150-180 minutos

---

## Objetivos de Aprendizaje

1. ✅ Comprender embeddings de Takens
2. ✅ Aplicar TDA a señales EEG
3. ✅ Detectar eventos mediante características topológicas
4. ✅ Clasificar estados cognitivos
5. ✅ Analizar señales fMRI con TDA

---

## 1. Introducción: Series Temporales en Neurociencias

### 1.1 Tipos de Señales

**EEG (Electroencefalografía):**
- Frecuencia de muestreo: 250-2000 Hz
- Resolución temporal: Milisegundos
- Resolución espacial: Baja (electrodos en cuero cabelludo)
- Aplicación: Detección de eventos rápidos, epilepsia, sueño

**fMRI (Resonancia Magnética Funcional):**
- Frecuencia de muestreo: ~0.5-2 Hz (TR = 0.5-2s)
- Resolución temporal: Segundos
- Resolución espacial: Alta (mm³)
- Aplicación: Conectividad funcional, mapeo cerebral

### 1.2 ¿Por qué TDA para Series Temporales?

Ventajas de TDA:
- **Invarianza:** Robusto a ruido y amplitud
- **No linealidad:** Captura dinámicas complejas
- **Multi-escala:** Analiza diferentes frecuencias simultáneamente
- **Características interpretables:** Ciclos, persistencia

---

In [ ]:
# Importaciones
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import warnings
warnings.filterwarnings('ignore')

# TDA
from ripser import ripser
from persim import plot_diagrams
from gtda.time_series import takens_embedding_optimal_parameters

# Procesamiento de señales
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import pandas as pd

np.random.seed(42)
print("✅ Bibliotecas importadas")

---

## 2. Teorema de Takens y Embeddings

### 2.1 El Problema

Tenemos una **serie temporal 1D**, pero queremos:
- Aplicar TDA (necesita datos multi-dimensionales)
- Reconstruir la dinámica subyacente del sistema

### 2.2 Teorema de Takens (1981)

Dada una serie temporal escalar $x(t)$, podemos reconstruir el **espacio de estados** usando:

$$\mathbf{y}(t) = [x(t), x(t+\tau), x(t+2\tau), ..., x(t+(d-1)\tau)]$$

Donde:
- **τ (tau):** Delay (retraso)
- **d:** Dimensión de embedding

**Resultado:** Si $d \geq 2D + 1$ (donde $D$ es la dimensión del atractor), la reconstrucción preserva propiedades topológicas.

### 2.3 Selección de Parámetros

- **τ:** Primer mínimo de auto-información mutua o primer cero de autocorrelación
- **d:** Método de falsos vecinos más cercanos

---

## 3. Implementación: Takens Embedding

In [ ]:
def takens_embedding(timeseries, delay=1, dimension=3):
    """
    Crea embedding de Takens de una serie temporal.
    
    Parameters:
    -----------
    timeseries : np.ndarray
        Serie temporal 1D
    delay : int
        Delay τ
    dimension : int
        Dimensión de embedding
    
    Returns:
    --------
    np.ndarray : Puntos en espacio de embedding (n_points x dimension)
    """
    n = len(timeseries)
    m = n - (dimension - 1) * delay
    
    if m <= 0:
        raise ValueError("Serie temporal muy corta para estos parámetros")
    
    embedded = np.zeros((m, dimension))
    
    for i in range(dimension):
        start = i * delay
        end = start + m
        embedded[:, i] = timeseries[start:end]
    
    return embedded


def estimate_delay(timeseries, max_delay=100):
    """
    Estima delay óptimo usando primer mínimo de autocorrelación.
    """
    autocorr = np.correlate(timeseries - np.mean(timeseries), 
                           timeseries - np.mean(timeseries), 
                           mode='full')
    autocorr = autocorr[len(autocorr)//2:]
    autocorr = autocorr / autocorr[0]
    
    # Primer cruce por cero o mínimo local
    for i in range(1, min(max_delay, len(autocorr)-1)):
        if autocorr[i] < autocorr[i-1] and autocorr[i] < autocorr[i+1]:
            return i
    
    # Si no hay mínimo, usar 1/e
    threshold = 1/np.e
    for i in range(1, min(max_delay, len(autocorr))):
        if autocorr[i] < threshold:
            return i
    
    return 1


print("✅ Funciones de Takens embedding definidas")

### Ejemplo: Sistema de Lorenz

In [ ]:
# Generar sistema de Lorenz (caótico)
def lorenz(xyz, t, sigma=10, rho=28, beta=8/3):
    x, y, z = xyz
    return [
        sigma * (y - x),
        x * (rho - z) - y,
        x * y - beta * z
    ]

from scipy.integrate import odeint

t = np.linspace(0, 50, 5000)
xyz0 = [1.0, 1.0, 1.0]
xyz = odeint(lorenz, xyz0, t)

# Usar solo componente x (como si fuera señal observada)
x_signal = xyz[:, 0]

print(f"📊 Señal de Lorenz generada: {len(x_signal)} puntos")

# Estimar delay
delay_opt = estimate_delay(x_signal)
print(f"\n⏱️  Delay óptimo estimado: τ = {delay_opt}")

# Crear embedding
embedded = takens_embedding(x_signal, delay=delay_opt, dimension=3)
print(f"✅ Embedding creado: {embedded.shape}")

# Visualizar
fig = plt.figure(figsize=(18, 5))

# 1. Serie temporal original
ax1 = fig.add_subplot(131)
ax1.plot(t[:1000], x_signal[:1000], linewidth=1)
ax1.set_xlabel('Tiempo', fontsize=11)
ax1.set_ylabel('x(t)', fontsize=11)
ax1.set_title('Serie Temporal: Componente x de Lorenz', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

# 2. Atractor verdadero (3D)
ax2 = fig.add_subplot(132, projection='3d')
ax2.plot(xyz[:, 0], xyz[:, 1], xyz[:, 2], linewidth=0.5, alpha=0.7)
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('z')
ax2.set_title('Atractor Verdadero (x, y, z)', fontsize=12, fontweight='bold')

# 3. Atractor reconstruido (Takens)
ax3 = fig.add_subplot(133, projection='3d')
ax3.plot(embedded[:, 0], embedded[:, 1], embedded[:, 2], 
         linewidth=0.5, alpha=0.7, color='red')
ax3.set_xlabel(f'x(t)')
ax3.set_ylabel(f'x(t+{delay_opt})')
ax3.set_zlabel(f'x(t+{2*delay_opt})')
ax3.set_title(f'Atractor Reconstruido (Takens, τ={delay_opt})', 
             fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 El embedding de Takens reconstruye la geometría del atractor!")
print("   Compara el atractor verdadero vs reconstruido - ¡son topológicamente equivalentes!")

---

## 4. Generación de Señales EEG Sintéticas

In [ ]:
def generate_eeg_signal(duration=10, fs=250, state='normal'):
    """
    Genera señal EEG sintética.
    
    Parameters:
    -----------
    duration : float
        Duración en segundos
    fs : int
        Frecuencia de muestreo (Hz)
    state : str
        'normal', 'seizure' (crisis epiléptica), 'sleep'
    """
    n_samples = int(duration * fs)
    t = np.linspace(0, duration, n_samples)
    
    if state == 'normal':
        # Ritmos normales: alpha (8-13 Hz), beta (13-30 Hz)
        alpha = 0.5 * np.sin(2 * np.pi * 10 * t)  # 10 Hz
        beta = 0.3 * np.sin(2 * np.pi * 20 * t)   # 20 Hz
        noise = 0.2 * np.random.randn(n_samples)
        eeg = alpha + beta + noise
        
    elif state == 'seizure':
        # Crisis: spike-wave a 3 Hz, alta amplitud
        spike_wave = 2.0 * np.sin(2 * np.pi * 3 * t)
        harmonics = 0.5 * np.sin(2 * np.pi * 6 * t)
        noise = 0.1 * np.random.randn(n_samples)
        eeg = spike_wave + harmonics + noise
        
    elif state == 'sleep':
        # Sueño: delta (0.5-4 Hz), alta amplitud
        delta = 1.5 * np.sin(2 * np.pi * 2 * t)
        theta = 0.3 * np.sin(2 * np.pi * 6 * t)  # 6 Hz
        noise = 0.15 * np.random.randn(n_samples)
        eeg = delta + theta + noise
    
    return t, eeg


# Generar señales de diferentes estados
print("🧠 Generando señales EEG sintéticas...\n")

states = ['normal', 'seizure', 'sleep']
eeg_signals = {}

for state in states:
    t, eeg = generate_eeg_signal(duration=10, fs=250, state=state)
    eeg_signals[state] = (t, eeg)
    print(f"✅ {state.capitalize()}: {len(eeg)} muestras")

# Visualizar señales
fig, axes = plt.subplots(3, 2, figsize=(16, 10))

for idx, state in enumerate(states):
    t, eeg = eeg_signals[state]
    
    # Serie temporal
    ax1 = axes[idx, 0]
    ax1.plot(t[:1000], eeg[:1000], linewidth=1)
    ax1.set_xlabel('Tiempo (s)', fontsize=10)
    ax1.set_ylabel('Amplitud (μV)', fontsize=10)
    ax1.set_title(f'EEG: {state.capitalize()}', fontsize=11, fontweight='bold')
    ax1.grid(True, alpha=0.3)
    
    # Espectro de frecuencia
    ax2 = axes[idx, 1]
    freqs = fftfreq(len(eeg), 1/250)
    fft_vals = np.abs(fft(eeg))
    
    # Solo frecuencias positivas hasta 50 Hz
    mask = (freqs >= 0) & (freqs <= 50)
    ax2.plot(freqs[mask], fft_vals[mask], linewidth=1.5)
    ax2.set_xlabel('Frecuencia (Hz)', fontsize=10)
    ax2.set_ylabel('Potencia', fontsize=10)
    ax2.set_title(f'Espectro: {state.capitalize()}', fontsize=11, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    # Marcar bandas de frecuencia
    ax2.axvspan(0.5, 4, alpha=0.2, color='purple', label='Delta')
    ax2.axvspan(8, 13, alpha=0.2, color='green', label='Alpha')
    ax2.axvspan(13, 30, alpha=0.2, color='orange', label='Beta')
    if idx == 0:
        ax2.legend(loc='upper right', fontsize=8)

plt.tight_layout()
plt.show()

---

## 5. Análisis Topológico de EEG

In [ ]:
# Aplicar Takens y TDA a cada señal
print("⏳ Aplicando Takens embedding y TDA a señales EEG...\n")

eeg_embeddings = {}
eeg_diagrams = {}

for state in states:
    t, eeg = eeg_signals[state]
    
    # Normalizar
    eeg_norm = zscore(eeg)
    
    # Estimar parámetros
    delay = estimate_delay(eeg_norm, max_delay=50)
    
    # Embedding
    embedded = takens_embedding(eeg_norm, delay=delay, dimension=3)
    eeg_embeddings[state] = embedded
    
    # TDA
    result = ripser(embedded, maxdim=1, thresh=5.0)
    eeg_diagrams[state] = result['dgms']
    
    print(f"✅ {state.capitalize()}:")
    print(f"   Delay: τ = {delay}")
    print(f"   Embedding: {embedded.shape}")
    print(f"   H₁ (ciclos): {len(result['dgms'][1])}\n")

# Visualizar embeddings y diagramas
fig = plt.figure(figsize=(18, 10))

for idx, state in enumerate(states):
    # Embedding 3D
    ax1 = fig.add_subplot(2, 3, idx+1, projection='3d')
    emb = eeg_embeddings[state]
    ax1.plot(emb[:1000, 0], emb[:1000, 1], emb[:1000, 2], 
            linewidth=0.5, alpha=0.7)
    ax1.set_title(f'Embedding: {state.capitalize()}', 
                 fontsize=11, fontweight='bold')
    ax1.set_xlabel('x(t)')
    ax1.set_ylabel('x(t+τ)')
    ax1.set_zlabel('x(t+2τ)')
    
    # Diagrama de persistencia
    ax2 = fig.add_subplot(2, 3, idx+4)
    plot_diagrams(eeg_diagrams[state], ax=ax2)
    ax2.set_title(f'Persistencia: {state.capitalize()}\nH₁={len(eeg_diagrams[state][1])} ciclos',
                 fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Observaciones:")
print("   • Estado Normal: Embedding complejo, múltiples ciclos")
print("   • Seizure: Estructura más simple y regular (3 Hz dominante)")
print("   • Sleep: Ciclos lentos y persistentes (ondas delta)")

---

## 6. Clasificación de Estados usando TDA

In [ ]:
def extract_tda_features_from_signal(signal, fs=250):
    """
    Extrae características topológicas de una señal temporal.
    """
    # Normalizar
    signal_norm = zscore(signal)
    
    # Embedding
    delay = estimate_delay(signal_norm, max_delay=30)
    embedded = takens_embedding(signal_norm, delay=delay, dimension=3)
    
    # TDA
    result = ripser(embedded, maxdim=1, thresh=5.0)
    dgm1 = result['dgms'][1]
    
    # Características
    features = {}
    
    if len(dgm1) > 0:
        dgm1_finite = dgm1[np.isfinite(dgm1[:, 1])]
        if len(dgm1_finite) > 0:
            lifetimes = dgm1_finite[:, 1] - dgm1_finite[:, 0]
            features['n_cycles'] = len(dgm1_finite)
            features['max_persistence'] = np.max(lifetimes)
            features['mean_persistence'] = np.mean(lifetimes)
            features['std_persistence'] = np.std(lifetimes)
            features['total_persistence'] = np.sum(lifetimes)
        else:
            features = {k: 0 for k in ['n_cycles', 'max_persistence', 
                                      'mean_persistence', 'std_persistence', 
                                      'total_persistence']}
    else:
        features = {k: 0 for k in ['n_cycles', 'max_persistence', 
                                  'mean_persistence', 'std_persistence', 
                                  'total_persistence']}
    
    # Características tradicionales
    features['mean_amplitude'] = np.mean(np.abs(signal))
    features['std_amplitude'] = np.std(signal)
    
    # Características espectrales
    freqs = fftfreq(len(signal), 1/fs)
    fft_vals = np.abs(fft(signal))
    
    # Potencia en bandas
    delta_mask = (freqs >= 0.5) & (freqs <= 4)
    alpha_mask = (freqs >= 8) & (freqs <= 13)
    beta_mask = (freqs >= 13) & (freqs <= 30)
    
    features['delta_power'] = np.sum(fft_vals[delta_mask])
    features['alpha_power'] = np.sum(fft_vals[alpha_mask])
    features['beta_power'] = np.sum(fft_vals[beta_mask])
    
    return features


# Generar dataset de entrenamiento
print("📊 Generando dataset de clasificación...\n")

n_samples_per_class = 100
X_data = []
y_data = []

for class_idx, state in enumerate(states):
    print(f"⏳ Generando {n_samples_per_class} muestras de '{state}'...")
    for i in range(n_samples_per_class):
        _, eeg = generate_eeg_signal(duration=5, fs=250, state=state)
        features = extract_tda_features_from_signal(eeg, fs=250)
        X_data.append(list(features.values()))
        y_data.append(class_idx)

X = np.array(X_data)
y = np.array(y_data)

print(f"\n✅ Dataset creado: {X.shape}")
print(f"   Características: {list(features.keys())}")

# Dividir en train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Entrenar clasificador
print("\n🎯 Entrenando clasificador Random Forest...")
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluar
y_pred = clf.predict(X_test)
accuracy = np.mean(y_pred == y_test)

print(f"\n✅ Precisión en test: {accuracy:.1%}")
print("\n📊 Reporte de clasificación:\n")
print(classification_report(y_test, y_pred, 
                          target_names=states))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=states, yticklabels=states, ax=ax)
ax.set_xlabel('Predicho', fontsize=12)
ax.set_ylabel('Verdadero', fontsize=12)
ax.set_title(f'Matriz de Confusión\n(Precisión: {accuracy:.1%})', 
            fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Importancia de características
feature_names = list(features.keys())
importances = clf.feature_importances_
indices = np.argsort(importances)[::-1]

fig, ax = plt.subplots(figsize=(12, 6))
ax.barh(range(len(importances)), importances[indices], alpha=0.7)
ax.set_yticks(range(len(importances)))
ax.set_yticklabels([feature_names[i] for i in indices])
ax.set_xlabel('Importancia', fontsize=12)
ax.set_title('Importancia de Características TDA para Clasificación', 
            fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

print("\n💡 Características más importantes:")
for i in range(5):
    idx = indices[i]
    print(f"   {i+1}. {feature_names[idx]}: {importances[idx]:.3f}")

---

## 7. Resumen y Conclusiones

### ✅ Lo que aprendimos:

1. **Teorema de Takens:** Reconstrucción de dinámica desde serie 1D
2. **Embeddings:** Transformar series temporales en nubes de puntos
3. **TDA en señales:** Aplicar homología persistente a EEG
4. **Clasificación:** Usar características topológicas para ML
5. **Aplicaciones:** Detección de crisis, estados de sueño, etc.

### 🧠 Aplicaciones Clínicas:

- **Epilepsia:** Detección temprana de crisis (cambios topológicos)
- **Sueño:** Clasificación automática de etapas
- **Anestesia:** Monitoreo de profundidad
- **Coma:** Evaluación de nivel de consciencia
- **Cognición:** Análisis de estados atencionales

### 🔑 Ventajas de TDA:

- **Robustez:** Invariante a ruido y amplitud
- **Interpretabilidad:** Ciclos tienen significado físico
- **Multi-escala:** Captura múltiples frecuencias
- **No linealidad:** Detecta dinámicas complejas

---

## Referencias

1. Takens, F. (1981). "Detecting strange attractors in turbulence"
2. Perea et al. (2015). "Sliding windows and persistence for time series"
3. Myers et al. (2019). "Persistent homology of complex networks for dynamic state detection"
4. Khalid et al. (2020). "TDA for EEG signal analysis"

---

## 🎉 ¡Completaste todos los tutoriales!

Has dominado:
1. ✅ Fundamentos de TDA
2. ✅ Homología persistente avanzada
3. ✅ Conectividad cerebral
4. ✅ Algoritmo Mapper
5. ✅ Series temporales neuronales

**Próximos pasos:**
- Aplica estos métodos a datos reales
- Explora bibliotecas adicionales (Giotto-TDA, GUDHI)
- Lee papers de investigación
- ¡Contribuye al campo!

---

**Autor:** MARK-126  
**Licencia:** MIT